# Basic end-to-end training of a LightGBM model

Features that are illustrated in this kernel:
- data reading with **memory footprint reduction**
- categorical feature encoding using **one-hot-encoding (OHE)**
- **under-sampling** to address imbalance of different categories in the data
- **gradient-boosted decision trees** using _**LightGBM**_ package
- **early stopping** in _**LightGBM**_ model training to avoid overtraining
- **learning rate decay** in _**LightGBM**_ model training to improve convergence to the minimum
- **hyperparameter optimisation** of the model using random search in cross validation
- submission preparation
This kernel inherited ideas and SW solutions from other public kernels and in such cases I will post direct references to the original product, that that you can get some additional insights from the source.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
PATH = "/home/mlisovyi/.kaggle/competitions/home-credit-default-risk/"
print(os.listdir(PATH))

# Any results you write to the current directory are saved as output.

['application_test.csv', 'bureau.csv', 'bureau_balance.csv', 'installments_payments.csv.zip', 'credit_card_balance.csv.zip', 'sample_submission.csv', 'credit_card_balance.csv', 'application_train.csv.zip', 'application_test.csv.zip', 'previous_application.csv', 'sample_submission.csv.zip', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'bureau_balance.csv.zip', 'application_train.csv', 'bureau.csv.zip', 'previous_application.csv.zip', 'POS_CASH_balance.csv.zip']


## Read in the data reducing memory pattern for variables.
The implementation was copied over from [this kernel](https://www.kaggle.com/gemartin/load-data-reduce-memory-usage)

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
application_train = import_data(PATH+'application_train.csv')
application_test = import_data(PATH+'application_test.csv')

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%
Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 9.40 MB
Decreased by 79.1%


## Categorical encoding
The function was taken from [this kernel](https://www.kaggle.com/sz8416/simple-intro-eda-baseline-model-with-gridsearch). It allows to do OneHotEncoding (OHE) keeping only those columns that are common to train and test samples. OHE is performed using `pd.get_dummies`, which allows to convert categorical features, while keeping numerical untouched

In [4]:
# use this if you want to convert categorical features to dummies(default)
def cat_to_dummy(train, test):
    train_d = pd.get_dummies(train, drop_first=True)
    test_d = pd.get_dummies(test, drop_first=True)
    # make sure that the number of features in train and test should be same
    for i in train_d.columns:
        if i not in test_d.columns:
            if i!='TARGET':
                train_d = train_d.drop(i, axis=1)
    for j in test_d.columns:
        if j not in train_d.columns:
            if j!='TARGET':
                test_d = test_d.drop(i, axis=1)
    print('Memory usage of train increases from {:.2f} to {:.2f} MB'.format(train.memory_usage().sum() / 1024**2, 
                                                                            train_d.memory_usage().sum() / 1024**2))
    print('Memory usage of test increases from {:.2f} to {:.2f} MB'.format(test.memory_usage().sum() / 1024**2, 
                                                                            test_d.memory_usage().sum() / 1024**2))
    return train_d, test_d
application_train_ohe, application_test_ohe = cat_to_dummy(application_train, application_test)

Memory usage of train increases from 59.54 to 90.33 MB
Memory usage of test increases from 9.40 to 14.27 MB


## Deal with category imbalance
Use a standard library to to random undersampling on the dominating category

In [5]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=314)
X_rus, y_rus = rus.fit_sample(application_train_ohe.drop(['SK_ID_CURR', 'TARGET'], axis=1).fillna(-1), 
                              application_train_ohe['TARGET'])

# Model fitting with HyperParameter optimisation
We will use LightGBM classifier - LightGBM allows to build very sophysticated models with a very short training time.
### Split the full sample into train/test (80/20)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.20, random_state=314, stratify=y_rus)

### Prepare learning rate shrinkage

In [7]:
def learning_rate_decay_power(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

### Use test subset for early stopping criterion 
This allows us to avoid overtraining and we do not need to optimise the number of trees

In [8]:
import lightgbm as lgb
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_decay_power)],
            'verbose': 100}

### Set up HyperParameter search
We use random search, which is more flexible and more efficient than a grid search

In [9]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [10]:
#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 20

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 1000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=1000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=5,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
gs.fit(X_train, y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.7486
[200]	valid's auc: 0.750586
Early stopping, best iteration is:
[238]	valid's auc: 0.750898
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.748301
[200]	valid's auc: 0.750963
[300]	valid's auc: 0.751416
[400]	valid's auc: 0.751612
Early stopping, best iteration is:
[408]	valid's auc: 0.751623
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.74901
[200]	valid's auc: 0.751554
[300]	valid's auc: 0.752223
[400]	valid's auc: 0.752392
[500]	valid's auc: 0.752453
Early stopping, best iteration is:
[532]	valid's auc: 0.752477
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.74819
[200]	valid's auc: 0.750677
[300]	valid's auc: 0.751216
[400]	valid's auc: 0.751432
Early stopping, best iteration is:
[400]	valid's auc: 0.751432
Training until validat

Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749065
Early stopping, best iteration is:
[121]	valid's auc: 0.74944
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749779
Early stopping, best iteration is:
[153]	valid's auc: 0.75096
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[84]	valid's auc: 0.750011
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749727
Early stopping, best iteration is:
[127]	valid's auc: 0.750346
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749296
Early stopping, best iteration is:
[164]	valid's auc: 0.750917
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749326
Early stopping, best iteration is:
[177]	valid's auc: 0.751192
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.749936
Early stopping, best it

[400]	valid's auc: 0.749418
Early stopping, best iteration is:
[479]	valid's auc: 0.749514
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.747492
[200]	valid's auc: 0.750195
[300]	valid's auc: 0.750874
[400]	valid's auc: 0.751138
[500]	valid's auc: 0.751243
[600]	valid's auc: 0.751302
[700]	valid's auc: 0.751359
[800]	valid's auc: 0.751429
Early stopping, best iteration is:
[857]	valid's auc: 0.751472
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.747308
[200]	valid's auc: 0.749674
[300]	valid's auc: 0.750341
Early stopping, best iteration is:
[328]	valid's auc: 0.750424
Training until validation scores don't improve for 10 rounds.
[100]	valid's auc: 0.747968
[200]	valid's auc: 0.750228
[300]	valid's auc: 0.750877
[400]	valid's auc: 0.751131
[500]	valid's auc: 0.751241
[600]	valid's auc: 0.751321
[700]	valid's auc: 0.751382
[800]	valid's auc: 0.751453
[900]	valid's auc: 0.751526
Early stopping, best iteration is:
[9

### Look at the performance of the top-5 parameter choices
(the list is inverted)

In [ ]:
print("Valid+-Std     Train  :   Parameters")
for i in np.argsort(gs.cv_results_['mean_test_score'])[-5:]:
    print('{1:.3f}+-{3:.3f}     {2:.3f}   :  {0}'.format(gs.cv_results_['params'][i], 
                                    gs.cv_results_['mean_test_score'][i], 
                                    gs.cv_results_['mean_train_score'][i],
                                    gs.cv_results_['std_test_score'][i]))

### Plot feature importance

In [ ]:
feat_imp = pd.Series(gs.best_estimator_.feature_importances_, index=application_train_ohe.drop(['SK_ID_CURR', 'TARGET'], axis=1).columns)
feat_imp.nlargest(10).plot(kind='barh')

## Build the final model
We do training with the 0.8 subset of the dataset and 0.2 subset for early stopping. We use the tuned parameter values but a smaller learning rate to allow smoother convergence to the minimum

In [ ]:
clf_final = lgb.LGBMClassifier(**gs.best_estimator_.get_params())
#force larger number of max trees and smaller learning rate
clf_final.set_params(n_estimators=5000, learning_rate=0.005)
clf_final.fit(X_train, y_train, **fit_params)

# Predict on the submission test sample

In [ ]:
probabilities = gs.best_estimator_.predict_proba(application_test_ohe.drop(['SK_ID_CURR'], axis=1).fillna(-1))
submission = pd.DataFrame({
    'SK_ID_CURR': application_test_ohe['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
submission.to_csv("submission.csv", index=False)